<a href="https://colab.research.google.com/github/hrisk/master_thesis/blob/main/HRNet_for_pose_detection_on_JRDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pose Detection with HRNet


## Install HRNet

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/lxy5513/hrnet.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q {git_repo_url}
  #!cd {project_name} && pip install -q -r requirements.txt
  !pip install -q youtube-dl EasyDict==1.7 Cython pyyaml json_tricks scikit-image yacs>=0.1.5 ipdb
  !cd {project_name}/lib && make
  
import sys
sys.path.append(project_name)
sys.path.append(join(project_name, 'tools'))
sys.path.append(join(project_name, 'lib'))
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

from IPython.display import YouTubeVideo

ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.19 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.25.0 which is incompatible.
cd nms; python setup_linux.py build_ext --inplace; rm -rf build; cd ../../
running build_ext
cythoning cpu_nms.pyx to cpu_nms.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/hrnet/lib/nms/cpu_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
skipping 'gpu_nms.cpp' Cython extension (up-to-date)
building 'cpu_nms' extension
creating build
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-OGiuun/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-sec

## Download pretrained models

In [ ]:
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id={file_id}" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm={confirm_text}&id={file_id}" -o {file_name}
  

pretrained_model_parent_directory = 'models/pytorch/pose_coco'
pretrained_model_path = join(project_name, pretrained_model_parent_directory, 'pose_hrnet_w48_256x192.pth')
if not exists(pretrained_model_path):
  # download the pretrained model
  !rm -rf {project_name}/models
  !mkdir -p {project_name}/{pretrained_model_parent_directory}
  download_from_google_drive('15T2XqPjW7Ex0uyC1miGVYUv7ULOxIyJI', pretrained_model_path)
  
yolo_pretrained_model_path = join(project_name, 'lib/detector/yolo/yolov3.weights')
if not exists(yolo_pretrained_model_path):
  # download the YOLO weights
  !cd {project_name}/lib/detector/yolo/ && wget -q https://pjreddie.com/media/files/yolov3.weights

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   2060      0 --:--:-- --:--:-- --:--:--  2060
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  243M    0  243M    0     0   115M      0 --:--:--  0:00:02 --:--:--  189M


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Execute pose detection on the first 5 seconds:

In [ ]:
# execute HRNet on video frames
import cv2
import numpy as np
from utilitys import plot_keypoint, PreProcess
from lib.core.inference import get_final_preds
import torch
import config
import models
from config import cfg, update_config
from types import SimpleNamespace


update_config(cfg,
              SimpleNamespace(cfg = join(project_name, 'experiments/coco/hrnet/w48_256x192_adam_lr1e-3.yaml'),
                              opts = '',
                              modelDir='', logDir='', dataDir='', prevModelDir=''))
def model_load(config):
  model = models.pose_hrnet.get_pose_net(config, is_train=False)
  model_file_name  = join(project_name, 'models/pytorch/pose_coco/pose_hrnet_w48_256x192.pth')
  state_dict = torch.load(model_file_name)
  from collections import OrderedDict
  new_state_dict = OrderedDict()
  for k, v in state_dict.items():
      name = k # remove module.
      #  print(name,'\t')
      new_state_dict[name] = v
  model.load_state_dict(new_state_dict)
  model.eval()
  return model
pose_model = model_load(cfg).cuda()

from lib.detector.yolo.human_detector import human_bbox_get as yolo_det
from lib.detector.yolo.human_detector import load_model as yolo_model
human_model = yolo_model()
data = []
for im in os.listdir("/content/drive/MyDrive/Minor_Thesis/tressider-2019-03-16_0"):
  d = {"image_id": im}
  try:
    # TODO: Yolo needs RGB?
    frame = cv2.imread(f'/content/drive/MyDrive/Minor_Thesis/tressider-2019-03-16_0/{im}')
    bboxs, scores = yolo_det(frame, human_model)
    # bbox is coordinate location
    inputs, origin_img, center, scale = PreProcess(frame, bboxs, scores, cfg)

    with torch.no_grad():
      # compute output heatmap
      inputs = inputs[:,[2,1,0]]
      output = pose_model(inputs.cuda())
      # compute coordinate
      preds, maxvals = get_final_preds(
          cfg, output.clone().cpu().numpy(), np.asarray(center), np.asarray(scale))
      for i in range(bbox.shape[0]):
        d["bbox"] = bbox[i]
        d["keypoints"] = preds[i]
        data.append(d)

    image = plot_keypoint(origin_img, preds, maxvals, 0.1)
  except:
      print("no pose detected!")


Loading network.....
Network successfully loaded
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detected!
no pose detecte